<a href="https://colab.research.google.com/github/apetuhova/innopolis/blob/main/flask_server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flask приложение. Сервер.


1. Для тестирования работы модели запустить сервер в этом ноутбуке и выполнить команды в ноутбуке-клиенте.
2. Для тестирования загрузки csv-файла перейти по ссылке [адрес_сервера]/web

In [1]:
!pip install flask-ngrok
!pip install flask==0.12.2

     |████████████████████████████████| 83 kB 987 kB/s 
  Attempting uninstall: flask
    Found existing installation: Flask 1.1.4
    Uninstalling Flask-1.1.4:
      Successfully uninstalled Flask-1.1.4


In [2]:
%%writefile flask_app2.py
# Загружаем билиотеки
import flask
from flask import Flask, request, render_template, redirect,url_for, send_from_directory
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
import os
import numpy as np
from flask_ngrok import run_with_ngrok # эта строчка не нужна, если запускаете на своем ПК
from werkzeug.utils import secure_filename

# Задаем имя серверу
app = Flask(__name__)
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False
app.config['UPLOAD_FOLDER'] = '/content/'
ALLOWED_EXTENSIONS = {'csv'} #{'png','jpeg'}
run_with_ngrok(app) # эта строчка не нужна, если запускаете на своем ПК

# Загрузка модели

def predict_model(x,p):
  x=np.array(x)
  p=np.array(p)
  ypred=p[0]*x**3+p[1]*x**2+p[2]*x+p[3] 
  return ypred.tolist()

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def hello():
    return "Hello World!"


@app.route('/uploads/<name>')
def download_file(name):
  return send_from_directory(app.config['UPLOAD_FOLDER'],name)



@app.route('/web', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an
        # empty file without a filename.
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            #return redirect(url_for('download_file', name=filename))            

            df=pd.read_csv(filename).head(5).to_html()        
            return df #render_template('simple.html', tables=[df.to_html(classes='data')], titles=df.columns.values)

    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''


@app.route("/plot", methods=["GET","POST"])
def plot():
  
  if flask.request.method == 'POST':

    x = json.loads(flask.request.json)['x']
    y = json.loads(flask.request.json)['y']
    ypred = json.loads(flask.request.json)['ypred']
    
    plt.plot(x,y)
    plt.plot(x,ypred)
    plt.savefig('plot.png')
    return redirect(url_for('download_file', name='plot.png'))  


  else:
    return "Method should be POST"
    

# Задаем функцию Predict
@app.route("/predict", methods=["GET","POST"])
def predict():
  data = {"success": False}

  if flask.request.method == 'POST':

    x = json.loads(flask.request.json)['x']
    p = json.loads(flask.request.json)['p']
    
    data["prediction"] = predict_model(x,p)
    #Записываем статус в data["success"]
    data["success"] = True    
    #Возвращаем результат json format 
    return flask.jsonify(data) 

  else:
    return "Method should be POST"

@app.route("/train", methods=["GET","POST"])
def retraining():

  if flask.request.method == 'POST':

    x = json.loads(flask.request.json)['train_x']
    y = json.loads(flask.request.json)['train_y']
    deg=3
    p=np.polyfit(x,y,deg=deg).tolist()
    return (flask.jsonify({'p':p}))
        #flask.jsonify({'x':x}) #"mnist_new.h5"
  else:
    return "Method should be POST"

# Запускаем Сервер
if __name__ == '__main__':
    app.run() 


Writing flask_app2.py


In [4]:
!python flask_app2.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://f566-35-231-105-170.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [17/Sep/2021 08:34:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:34:38] "POST /train HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:34:38] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:36:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:36:09] "POST /train HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:36:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:16] "POST /train HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:31] "POST /train HTTP/1.1" 200 -
127.0.0.1 - - [17/Sep/2021 08:38:31] "POST /predict HTTP/1.1" 200 -
^C
